In [1]:
from sklearn.datasets import make_moons

X, y=make_moons(n_samples=10000, noise=0.4, random_state=42)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params={'max_leaf_nodes': list(range(2,100)), 'min_samples_split':[2,3,4]}
grid_search=GridSearchCV(DecisionTreeClassifier(random_state=42), params, cv=3, n_jobs=1, verbose=1)

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [12]:
grid_search.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)

In [13]:
#Accuracy score
from sklearn.metrics import accuracy_score

y_pred=grid_search.predict(X_test)
accuracy_score(y_test,y_pred)

0.8695

In [14]:
#RandomForestClassifier

In [15]:
#generate 1,000 subsets of the training set, each containing 100 instances selected randomly.
from sklearn.model_selection import ShuffleSplit

n_trees=1000
n_instances=100

mini_split=[]

rs=ShuffleSplit(n_splits=n_trees,test_size=len(X_train)-n_instances,random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train=X_train[mini_train_index]
    y_mini_train=y_train[mini_train_index]
    mini_split.append((X_mini_train,y_mini_train))

In [22]:
#Train one decision tree on each subset using the best hyperparameter obtain from GridSearchCV
from sklearn.base import clone

forest= [clone(grid_search.best_estimator_) for _ in range(n_trees)]                #Creates 1000 decision tree with GridSearch best estimator

accuracy_scores=[]

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_split):
    tree.fit(X_mini_train,y_mini_train)
    
    y_pred=tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test,y_pred))

In [24]:
import numpy as np
np.mean(accuracy_scores)

0.8054499999999999

In [30]:
#For each test instance produce prediction of 1000 decision trees and use the most frequent prediction

Y_pred=np.empty([n_trees,len(X_test)])

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index]=tree.predict(X_test)

In [39]:
from scipy.stats import mode

y_pred_majority_votes,n_votes=mode(Y_pred,axis=0)

array([1., 1., 0., ..., 0., 0., 0.])

In [40]:
#Evaluate accuracy of the model

accuracy_score(y_test,y_pred_majority_votes.reshape([-1]))

0.872